In [ ]:
import os
import math
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import math

In [ ]:
class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional=False):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.rnn = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )
        # Adjust the output dimension if bidirectional is used
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, x):
        # RNN forward pass
        # Input shape: [batch_size, seq_len, input_dim]
        out, _ = self.rnn(x)  # out: [batch_size, seq_len, hidden_dim]
        # Use the last hidden state for classification
        out = out[:, -1, :]  # [batch_size, hidden_dim]
        out = self.fc(out)  # [batch_size, num_classes]
        return out


In [ ]:
# Hyperparameters
hidden_dim = 64
num_layers = 2
bidirectional = True  # Set to False for unidirectional RNN
num_classes = 5
learning_rate = 0.001
epochs = 50
batch_size = 32

# Prepare Data
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Model Initialization
model = RNNClassifier(
    input_dim=3,  # From reformat_tensor output
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    bidirectional=bidirectional
).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
print("Training RNN model...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        features = reformat_tensor(features).to(device)  # Reshape for RNN
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for features, labels in test_loader:
        features = reformat_tensor(features).to(device)
        labels = labels.to(device)

        outputs = model(features)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")
